In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Read the CSV file
data = pd.read_csv('household_power_consumption.csv')

# Convert 'Date' column to datetime type
data['Date'] = pd.to_datetime(data['Date'] + ' ' + data['Time'], dayfirst=True)

# Ensure columns contain numeric values
data['Global_active_power'] = pd.to_numeric(data['Global_active_power'], errors='coerce')
data['Global_reactive_power'] = pd.to_numeric(data['Global_reactive_power'], errors='coerce')
data['Voltage'] = pd.to_numeric(data['Voltage'], errors='coerce')
data['Global_intensity'] = pd.to_numeric(data['Global_intensity'], errors='coerce')
data['Sub_metering_1'] = pd.to_numeric(data['Sub_metering_1'], errors='coerce')
data['Sub_metering_2'] = pd.to_numeric(data['Sub_metering_2'], errors='coerce')
data['Sub_metering_3'] = pd.to_numeric(data['Sub_metering_3'], errors='coerce')

# Remove rows with missing values
data.dropna(inplace=True)

# Aggregate data on a daily basis
daily_data = data.groupby(data['Date'].dt.date).agg({
    'Global_active_power': 'mean',
    'Global_reactive_power': 'mean',
    'Voltage': 'mean',
    'Global_intensity': 'mean',
    'Sub_metering_1': 'mean',
    'Sub_metering_2': 'mean',
    'Sub_metering_3': 'mean'
}).reset_index()

# Extract year, month, and day features
daily_data['Year'] = pd.to_datetime(daily_data['Date']).dt.year
daily_data['Month'] = pd.to_datetime(daily_data['Date']).dt.month
daily_data['Day'] = pd.to_datetime(daily_data['Date']).dt.day

# Drop the original 'Date' column
daily_data.drop(columns=['Date'], inplace=True)

# Split data into features (X) and target (y)
X = daily_data[['Year', 'Month', 'Day', 'Global_reactive_power', 'Voltage', 'Global_intensity', 'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3']]
y = daily_data['Global_active_power']

# Split the data into training and testing sets using train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

# Define hyperparameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_features': ['sqrt', 'log2'],
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Model Building (Random Forest Regressor)
rf = RandomForestRegressor()

# Grid Search
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, n_jobs=-1, verbose=0)

# Fit the grid search model
grid_search.fit(X_train, y_train)

# Best parameters
print("Best parameters found:")
print(grid_search.best_params_)

# Prediction and Evaluation
y_pred = grid_search.predict(X_test)

# Evaluation Metrics
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error (MSE):", mse)
rmse = np.sqrt(mse)
print("Root Mean Squared Error (RMSE):", rmse)
mean_actual = y_test.mean()
rae = np.mean(np.abs(y_test - y_pred)) / np.mean(np.abs(y_test - mean_actual))
print("Relative Absolute Error (RAE):", rae)


In [ ]:
# Visualization
plt.figure(figsize=(10, 6))

# Plot actual values from training set as dots
plt.scatter(X_train.index, y_train, label='Actual (Training)', marker='.', color='green')

# Plot predicted values from testing set as a line
plt.scatter(X_test.index, y_pred, label='Predicted (Testing)', marker='.', color='red')

# Customize x-axis labels to show only the years 2007 and 2008
tick_locations = []
tick_labels = []
for index, row in daily_data.iterrows():
    if row['Year'] == 2007 and row['Month'] == 1 and row['Day'] == 1:  # If it's the beginning of 2007
        tick_locations.append(index)
        tick_labels.append('2007')  # Add the year label
    elif row['Year'] == 2008 and row['Month'] == 1 and row['Day'] == 1:  # If it's the beginning of 2008
        tick_locations.append(index)
        tick_labels.append('2008')  # Add the year label

plt.xticks(tick_locations, tick_labels, rotation=45)

plt.title('Actual (Training) vs. Predicted (Testing) Global Active Power')
plt.xlabel('Year')
plt.ylabel('Global Active Power')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Visualization: Histograms of Actual and Predicted Global Active Power
plt.figure(figsize=(10, 6))

# Histogram of Actual Global Active Power
plt.hist(y_test, bins=30, alpha=0.5, color='blue', label='Actual')

# Histogram of Predicted Global Active Power
plt.hist(y_pred, bins=30, alpha=0.5, color='orange', label='Predicted')

plt.title('Histogram of Actual vs. Predicted Global Active Power')
plt.xlabel('Global Active Power')
plt.ylabel('Frequency')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Visualize pairs of hyperparameters using pairplot
warnings.filterwarnings("ignore", message="The figure layout has changed to tight", module="seaborn.axisgrid")

results = pd.DataFrame(grid_search.cv_results_)
sns.pairplot(results[['param_n_estimators', 'param_max_depth', 'param_min_samples_split', 'param_min_samples_leaf', 'mean_test_score']])

# Set the figure size explicitly
plt.gcf().set_size_inches(12, 12)

# Adjust the layout of the figure
plt.subplots_adjust(top=0.95, bottom=0.05, left=0.05, right=0.95, hspace=0.2, wspace=0.2)

# Show the plot
plt.show()

In [ ]:
# Calculate the correlation matrix
corr_matrix = daily_data.corr()

# Plot the heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title('Correlation Matrix Heatmap')
plt.show()


In [ ]:
# Visualize the grid search results using Seaborn
results = pd.DataFrame(grid_search.cv_results_)
sns.set_style("whitegrid")
plt.figure(figsize=(10, 6))

# Plot a heatmap for each hyperparameter combination
pivoted_data = results.pivot_table(index='param_n_estimators', columns='param_min_samples_split',
                                   values='mean_test_score')
ax = sns.heatmap(pivoted_data, cmap="YlGnBu", annot=True, fmt=".3f", cbar=False)
ax.set_title("Grid Search Results for n_estimators and min_samples_split")
ax.set_xlabel('min_samples_split')
ax.set_ylabel('n_estimators')

plt.tight_layout()
plt.show()